# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files

**Date**: October 7th 2025

**Student Name**: Eduardo Navarro Martínez

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

spark = SparkSession.builder \
    .appName("Lab 7 - Server Log Analysis") \
    .master("spark://87a7c16492c8:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")




Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 10:15:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
logsPath = "/opt/spark/work-dir/data/lab7Logs"

logsSchema = StructType([StructField("timestamp", TimestampType(), True), StructField("level", StringType(), True), StructField("code", StringType(), True)])

rawLogsDF = spark.readStream \
    .format("text") \
    .load(logsPath)

parsedLogsDF = rawLogsDF.select(
    to_timestamp(split(col("value"), ",").getItem(0), "yyyy-MM-dd HH:mm:ss").alias("timestamp"),
    split(col("value"), ",").getItem(1).alias("level"),
    split(col("value"), ",").getItem(2).alias("code")
).filter(col("level") == "ERROR")

query = parsedLogsDF.writeStream \
            .outputMode("append") \
            .format("console") \
            .start()
query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 